## Make the matricies from the raw data.

In [1]:
import pandas as pd
import numpy as np
from functions import create_matrix

In [2]:
Circos_DATA = pd.read_csv('Circos_DATA.csv')

In [3]:
df = Circos_DATA
environmental = Circos_DATA.columns[2:7]
social = Circos_DATA.columns[7:14]
group1 = list(df[environmental].sum().sort_values(ascending=False).index)
group2 = list(df[social].sum().sort_values(ascending=False).index)
df = df[group1 + group2]

### df structure

```
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Response ID                1434 non-null   object
 1   Code                       1434 non-null   int64 
 2   Weather/Climate            1434 non-null   int64 
 3   Hydrology                  1434 non-null   int64 
 4   Ecology                    1434 non-null   int64 
 5   Geomorphology              1434 non-null   int64 
 6   Land Use/Land Cover        1434 non-null   int64 
 7   Demographic                1434 non-null   int64 
 8   Socioeconomic              1434 non-null   int64 
 9   Social Capital & Identity  1434 non-null   int64 
 10  Human Built Capital        1434 non-null   int64 
 11  Institutional              1434 non-null   int64 
 12  Livelihood activities      1434 non-null   int64 
 13  Commodity Prices           1434 non-null   int64 
 ```

In [4]:
code=0

type={}

# Setup our lists of categories for mapping connections.
environmental = Circos_DATA.columns[2:7]
social = Circos_DATA.columns[7:14]

# Use the same groups for every subplot:
groups = [environmental, social]


#if code == 1:
#
#elif code == 2:
#    groups = [environmental, social]
#else:
#    groups = None

df, n_papers = create_matrix(Circos_DATA, code=code, groups=groups)

type[0] = 'All manuscripts (n={n})'.format(n=n_papers)
type[1] = 'Manuscripts investigating social impacts on environmental systems (n={n})'.format(n=n_papers)
type[2] = 'Manuscripts investigating environmental impacts on social systems (n={n})'.format(n=n_papers)
type[3] = 'Manuscripts investigating coupled systems dynamics (n={n})'.format(n=n_papers)



### Make the Plot (attempt using Plotly)

In [5]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from functions import check_data, get_ideogram_ends, map_data, make_ribbon_ends
PI = np.pi

In [6]:
# Determine the size of every arc in the ideogram, based on total number of manuscripts.
matrix = np.array(df, dtype=int)
L = check_data(matrix)

row_sum = [np.sum(matrix[k,:]) for k in range(L)]

#set the gap between two consecutive ideograms
gap=2*PI*0.005
ideogram_length=2*PI*np.asarray(row_sum)/sum(row_sum)-gap*np.ones(L)

# Get the end angular coordinates for each ideogram arc
ideo_ends = get_ideogram_ends(ideogram_length, gap)

In [7]:
# Labels and colors will be consistent across all figures
greens = [
    'rgba(237,248,251, 0.75)',
    'rgba(204,236,230, 0.75)',
    'rgba(153,216,201, 0.75)',
    'rgba(102,194,164, 0.75)',
    'rgba(65,174,118, 0.75)',
    'rgba(35,139,69, 0.75)',
    'rgba(0,88,36, 0.75)']

blues = [
    'rgba(241,238,246, 0.75)',
    'rgba(189,201,225, 0.75)',
    'rgba(116,169,207, 0.75)',
    'rgba(43,140,190, 0.75)',
    'rgba(4,90,141, 0.75)']

greys = ['rgba(189,189,189, 0.75)']*12

# This isn't necessary, since we've made the green and blue color lists the right size.
environmental_colors = blues[0:len(list(environmental))]
social_colors = greens[0:len(list(social))]

ideo_colors= environmental_colors[::-1] + social_colors[::-1]

labels = list(df.columns)

In [8]:
mapped_data = map_data(matrix, row_sum, ideogram_length)
idx_sort=np.argsort(mapped_data, axis=1)
ribbon_ends=make_ribbon_ends(mapped_data, ideo_ends,  idx_sort)

# Use the correct ribbon colors depending on the type of plot:
if code == 1 or code == 2:
    ribbon_color=[L*[ideo_colors[k]] for k in range(L)]
else:
    ribbon_color=[L*[greys[k]] for k in range(L)]


### Create a Layout for the figure

In [9]:
from functions import make_layout, make_ribbon, make_self_rel, invPerm

layout=make_layout(type[code], 800)


In [10]:
# these values are set after a few trials 
radii_sribb=[0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30]

In [11]:
ribbon_info=[]
for k in range(L):
    sigma=idx_sort[k]
    sigma_inv=invPerm(sigma)
    for j in range(k, L):
        if matrix[k][j]==0 and matrix[j][k]==0: continue
        eta=idx_sort[j]
        eta_inv=invPerm(eta)
        l=ribbon_ends[k][sigma_inv[j]]

        if j==k:
            # CHANGE THIS APPEND TO A SIMPLE ADDITION....
            #layout['shapes'].append(make_self_rel(l, 'rgb(175,175,175)' ,
            #                        ideo_colors[k], radius=radii_sribb[k]))
            layout['shapes'] = layout['shapes'] + (make_self_rel(l, 'rgb(175,175,175)' ,
                                    ideo_colors[k], radius=radii_sribb[k]),)
            z=0.9*np.exp(1j*(l[0]+l[1])/2)
            #the text below will be displayed when hovering the mouse over the ribbon
            text='There were {:d}'.format(matrix[k][k])+' manuscripts with both {k} and {k} data'.format(k=labels[k])
            ribbon_info.append(go.Scatter(x=[z.real],
                                       y=[z.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ideo_colors[k]),
                                       text=text,
                                       hoverinfo='text'
                                       )
                              )
        else:
            r=ribbon_ends[j][eta_inv[k]]
            zi=0.9*np.exp(1j*(l[0]+l[1])/2)
            zf=0.9*np.exp(1j*(r[0]+r[1])/2)
            #texti and textf are the strings that will be displayed when hovering the mouse 
            #over the two ribbon ends
            if code == 1:
                this_color = ribbon_color[j][k]
                texti='There were {:d}'.format(matrix[k][j])+' manuscripts with both {j} and {k} data'.format(
                    k=labels[k], j=labels[j])
                textf='There were {:d}'.format(matrix[k][j])+' manuscripts with both {j} and {k} data'.format(
                    k=labels[k], j=labels[j])
            elif code == 2:
                this_color = ribbon_color[k][j]
                texti='There were {:d}'.format(matrix[k][j])+' manuscripts with both {k} and {j} data'.format(
                    k=labels[k], j=labels[j])
                textf='There were {:d}'.format(matrix[k][j])+' manuscripts with both {k} and {j} data'.format(
                    k=labels[k], j=labels[j])
            
            else:
                # Tom likes the greens as default.
                texti='There were {:d}'.format(matrix[k][j])+' manuscripts with both {j} and {k} data'.format(
                    k=labels[k], j=labels[j])
                textf='There were {:d}'.format(matrix[k][j])+' manuscripts with both {j} and {k} data'.format(
                    k=labels[k], j=labels[j])
            
                this_color = ribbon_color[k][j]
            ribbon_info.append(go.Scatter(x=[zi.real],
                                       y=[zi.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=this_color),
                                       text=texti,
                                       hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                       y=[zf.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=this_color),
                                       text=textf,
                                       hoverinfo='text'
                                       )
                              )
            r=(r[1], r[0])#IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            #append the ribbon shape
#           layout['shapes'] = layout['shapes'] + (make_ribbon(l, r, 'rgb(175,175,175)' , ribbon_color[k][j]),)
            layout['shapes'] = layout['shapes'] + (make_ribbon(l, r, 'rgb(175,175,175)' , this_color),)




In [12]:
from functions import make_ideogram_arc, make_ideo_shape
ideograms=[]
ideo_labels=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    z_avg = make_ideogram_arc(1.3, [np.mean(ideo_ends[k]), np.mean(ideo_ends[k])])
    ideo_labels.append(dict(
            x=z_avg.real[0],
            y=z_avg.imag[0],
            xref="x",
            yref="y",
            text=labels[k],
            showarrow=False,
            font=dict(
                family="arial",
                size=18
            )
        ))
    m=len(z)
    n=len(zi)
    ideograms.append(go.Scatter(x=z.real,
                             y=z.imag,
                             mode='lines',
                             name=labels[k],
                             line=dict(color=ideo_colors[k], shape='spline', width=0.25),
                             text=labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
                             hoverinfo='text',
                             )
                     )


    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    layout['shapes'] = layout['shapes'] + (make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]),)

data = go.Data(ideograms+ribbon_info)
fig = go.Figure(data=data, layout=layout)

fig.update_layout(
    template="plotly_white",
    annotations=ideo_labels,
    width=800,
    height=800
)

#import plotly.offline as off
#off.init_notebook_mode()

py.iplot(fig, filename='chord-diagram-ses')


/Users/kellycaylor/opt/anaconda3/envs/ses_meta_analysis/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [13]:
for k in range(L):
    print('ideogram {k} has color {c}'.format(k=labels[k], c=ideo_colors[k]))

print(ideo_colors)

ideogram Land Use/Land Cover has color rgba(4,90,141, 0.75)
ideogram Ecology has color rgba(43,140,190, 0.75)
ideogram Geomorphology has color rgba(116,169,207, 0.75)
ideogram Weather/Climate has color rgba(189,201,225, 0.75)
ideogram Hydrology has color rgba(241,238,246, 0.75)
ideogram Livelihood activities has color rgba(0,88,36, 0.75)
ideogram Demographic has color rgba(35,139,69, 0.75)
ideogram Socioeconomic has color rgba(65,174,118, 0.75)
ideogram Institutional has color rgba(102,194,164, 0.75)
ideogram Human Built Capital has color rgba(153,216,201, 0.75)
ideogram Social Capital & Identity has color rgba(204,236,230, 0.75)
ideogram Commodity Prices has color rgba(237,248,251, 0.75)
['rgba(4,90,141, 0.75)', 'rgba(43,140,190, 0.75)', 'rgba(116,169,207, 0.75)', 'rgba(189,201,225, 0.75)', 'rgba(241,238,246, 0.75)', 'rgba(0,88,36, 0.75)', 'rgba(35,139,69, 0.75)', 'rgba(65,174,118, 0.75)', 'rgba(102,194,164, 0.75)', 'rgba(153,216,201, 0.75)', 'rgba(204,236,230, 0.75)', 'rgba(237,248,2

In [14]:
#from plotly.offline import init_notebook_mode
#import plotly.graph_objs as go

#init_notebook_mode(connected=True)

#data = go.Data(ribbon_info+ideograms)
#fig = go.Figure(data=data, layout=layout)

#py.plot(fig, filename='chord-diagram-ses', auto_open=True)